### Forest cover analysis
This notebooks calculates all forest cover statstics and outputs including change compared to the GLC_FCD30D data from 1990.

In [1]:
import geopandas as gpd
import numpy as np
import os
import pandas as pd
import shutil

from rasterio.warp import Resampling

from config import Config
from core.analysis import calc_lucc_stats, calc_luc_frac, create_change_map, reproject_align_raster, reproject_raster
from core.utils import get_mapping_from_csv

In [2]:
config = Config.Config()

In [3]:
fcs30_mapping = get_mapping_from_csv(config.luc_fcs30_legend_path, col_key="pixel", col_value="class")    
fcs30_l1_mapping = get_mapping_from_csv(config.luc_fcs30_legend_path, col_key="pixel", col_value="class_l1")    
topo_mapping = get_mapping_from_csv(config.topo_legend_path, col_key="pixel", col_value="class_l2")
topo_cmap = get_mapping_from_csv(config.topo_legend_path, col_key="pixel", col_value="color_rgba", convert_rgba=True)
pixel_area = config.fcs30_resolution**2 / 1e6

## Reproject and align raster files

In [4]:
if os.path.exists(config.analysis_forest_folder):
    shutil.rmtree(config.analysis_forest_folder)
os.makedirs(config.analysis_forest_folder)

#

In [5]:
# Reproject FCS30 raster to projected crs used for topo maps at its native resolution
reproject_raster(config.luc_fcs30_1990_path, config.raster_fcs30_1990_proj_path, config.output_crs, resolution=config.fcs30_resolution, resampling=Resampling.nearest)

In [6]:
# Reproject topo raster to same resolution and alignment as projected FCS30 raster
reproject_align_raster(config.raster_topo_l2_path, config.raster_fcs30_1990_proj_path, config.raster_topo_aligned_fcs30_path, resampling=Resampling.mode, colormap=topo_cmap)

Writing colormap


## Aggregate forest changes by study area

In [7]:
study_area_lao = gpd.read_file(config.study_area_lao_path).geometry[0]
study_area_svnm = gpd.read_file(config.study_area_svnm_path).geometry[0]
study_areas = gpd.GeoDataFrame({
    "region": ["Lao", "South Vietnam"],
    "geometry": [study_area_lao, study_area_svnm]
}, crs=config.output_crs)

In [8]:
res_forest_change = calc_lucc_stats(
    study_areas,
    index_cols="region",
    src_raster=config.raster_topo_aligned_fcs30_path,
    dst_raster=config.raster_fcs30_1990_proj_path,
    src_mapping=topo_mapping,
    dst_mapping=fcs30_mapping,
    src_class="forest",
    dst_class="Forest",
    pixel_area=pixel_area
)
res_forest_change.drop("geometry", axis=1).to_csv(config.forest_stats_study_area, index=False)
np.round(res_forest_change, 1)

,geometry,region,area,src_perc,dst_perc,change_perc_total_area,change_perc_src_area,src_pixels,dst_pixels,src_area,dst_area,change_area
region,,,,,,,,,,,,
Lao,"MULTIPOLYGON (((-2027355.81 3268113.142, -2027...",Lao,225542.7,81.3,66.5,-14.8,-18.2,250603020.0,250603020.0,183263.4,149973.5,-33289.9
South Vietnam,"MULTIPOLYGON (((-1749472.749 3100402.783, -174...",South Vietnam,141163.6,52.6,39.5,-13.1,-25.0,156848452.7,156848452.7,74298.2,55748.8,-18549.4


# Aggregate forest changes by map sheet

In [9]:
# Load map sheet index
ms_index = gpd.read_file(config.map_sheet_index_geo_overedge_path).to_crs(config.output_crs)
ms_index = ms_index[["key", "edition", "map_info_date", "legend_type", "geometry"]]

# Crop to study area outline removing parts of map sheets not in GADM outlines of Vietnam and Laos
study_area = gpd.read_file(config.study_area_path).geometry[0]
ms_index.geometry = ms_index.geometry.intersection(study_area).make_valid()
ms_index = ms_index[ms_index.geometry.is_empty == False]

In [10]:
# Calculate forest cover change stats per map sheet
ms_index = calc_lucc_stats(
    ms_index,
    index_cols="key",
    src_raster=config.raster_topo_aligned_fcs30_path,
    dst_raster=config.raster_fcs30_1990_proj_path,
    src_mapping=topo_mapping,
    dst_mapping=fcs30_mapping,
    src_class="forest",
    dst_class="Forest",
    pixel_area=pixel_area
)

ms_index["forest_loss"] = ms_index["change_area"].apply(lambda x: -x if x < 0 else 0)
ms_index["forest_gain"] = ms_index["change_area"].apply(lambda x: x if x > 0 else 0)

In [11]:
# Create forest loss transition raster
create_change_map(
    config.raster_topo_aligned_fcs30_path,
    config.raster_fcs30_1990_proj_path,
    output_path=config.raster_forest_loss_path,
    src_raster_vals=config.topo_forest_vals,
    dst_raster_exclude=config.fcs30_forest_vals
)

# Calculate forest loss transitions per map sheet
forest_loss_frac = calc_luc_frac(config.raster_forest_loss_path, ms_index, include_cols="key", mapping=fcs30_l1_mapping)

# Map back to map sheet index including color of most common transition for each map sheet
forest_loss_frac["maj_forest_loss_transition"] = forest_loss_frac.drop(["count", "Filled value"], axis=1).idxmax(axis=1)
ms_index["maj_forest_loss_transition"] = ms_index[["key"]].join(forest_loss_frac[["maj_forest_loss_transition"]])["maj_forest_loss_transition"]

legend_fcs30 = pd.read_csv(config.luc_fcs30_legend_path).drop_duplicates("class_l1")
color_mapping = {name: color for name, color in zip(legend_fcs30["class_l1"], legend_fcs30["color"])}
ms_index["maj_forest_loss_transition_color"] = ms_index["maj_forest_loss_transition"].map(color_mapping)
ms_index.head(3)

Output written to ../data/processed/analysis/forest/forest_loss_topo_fcs30_1990.tif


,edition,map_info_date,legend_type,geometry,key,area,src_perc,dst_perc,change_perc_total_area,change_perc_src_area,src_pixels,dst_pixels,src_area,dst_area,change_area,forest_loss,forest_gain,maj_forest_loss_transition,maj_forest_loss_transition_color
key,,,,,,,,,,,,,,,,,,,
50491,2-AMS,1965,6,"POLYGON ((-2856826.453 3833631.259, -2828043.3...",50491,499.506552,95.909533,56.878993,-39.030540,-40.695162,555007.280232,555007.280232,479.074403,284.114298,-194.960106,194.960106,0.0,Shrubland,#966400
50494,2-AMS,1965,6,"POLYGON ((-2856826.453 3833631.259, -2855673.8...",50494,330.000098,85.278115,40.171187,-45.106928,-52.893908,366666.775044,366666.775044,281.417863,132.564958,-148.852906,148.852906,0.0,Shrubland,#966400
50501,1-AMS,1965,6,"POLYGON ((-2830219.422 3861019.533, -2857293.8...",50501,148.816483,96.406453,44.138082,-52.268371,-54.216672,165351.648227,165351.648227,143.468694,65.684742,-77.783952,77.783952,0.0,Shrubland,#966400


In [12]:
ms_index[["maj_forest_loss_transition", "maj_forest_loss_transition_color"]].value_counts()

maj_forest_loss_transition  maj_forest_loss_transition_color
Shrubland                   #966400                             619
Rainfed cropland            #ffff64                             141
Irrigated cropland          #aaf0f0                              69
Impervious surface          #c31400                              17
Water body                  #0046c8                               3
Bare areas                  #fff5d7                               2
Coastal wetland             #f57ab6                               1
Name: count, dtype: int64

In [13]:
ms_index.to_file(config.forest_stats_map_sheets, driver="GeoJSON", index=False)

/home/s1835403/datastore/.conda/envs/topo-maps/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: NaN of Infinity value found. Skipped
  ogr_write(
